In [ ]:
#########################################################################################
# Read secretes from Azure Key Vault
#########################################################################################

key_vault = "https://yourkeyvalut.vault.azure.net/"
tenant = mssparkutils.credentials.getSecret(key_vault , "FabricTenantId")
client = mssparkutils.credentials.getSecret(key_vault , "FabricClientId")
client_secret = mssparkutils.credentials.getSecret(key_vault , "FabricClientSecret")

In [ ]:

#########################################################################################
# Authentication - Replace string variables with your relevant values       
#########################################################################################
 
import json, requests, pandas as pd
import datetime
 
try:
    from azure.identity import ClientSecretCredential
except Exception:
     !pip install azure.identity
     from azure.identity import ClientSecretCredential
 


In [ ]:
# Generates the access token for the Service Principal
api = 'https://analysis.windows.net/powerbi/api/.default'
auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/',
                              tenant_id = tenant,
                              client_id = client,
                              client_secret = client_secret)
access_token = auth.get_token(api)
access_token = access_token.token
 
print('\nSuccessfully authenticated.')  

In [ ]:
 
 
#########################################################################################
# Get Refreshables
#########################################################################################

base_urls = {'powerbi':'https://api.powerbi.com/v1.0/myorg/', 'fabric':'https://api.fabric.microsoft.com/v1/'}
base_url = 'https://api.powerbi.com/v1.0/myorg/'
fabric_url='https://api.fabric.microsoft.com/v1/'
header = {'Authorization': f'Bearer {access_token}'}

fabric_paths={'workspace':"workspaces"}
powerbi_paths={'groups':"groups", 'datasets':"datasets", 'adminReport':"admin/reports", 'refreshables':"admin/capacities/refreshables"}



In [ ]:
from pyspark.sql.functions import *
import requests, os, datetime
from delta.tables import *

In [ ]:
def get_response(base_url, relative_path):
    response=requests.get(base_url + relative_path, headers=header)
    return response

def response_to_df(response):
    contentRDD = spark.sparkContext.parallelize([response.content.decode()])
    dfContent = spark.read.json(contentRDD)
    return dfContent

def createTable(dfResponse, tableName):
    ## write the spark dataframe out to a delta table
    if spark.catalog.tableExists(tableName):
            #get existing table
            targetTable = DeltaTable.forName(spark,tableName)
            #merge into table
            (targetTable.alias("t")
                .merge(dfResponse.alias("s"), "s.Id = t.Id")
                .whenMatchedUpdateAll()
                .whenNotMatchedInsertAll()
                .execute()
            )
    else:
        # create workspace table
        dfResponse.write.mode('overwrite').format("delta").saveAsTable(tableName)

In [ ]:
#########################################################################################
# For each power bi path, generate tables
#########################################################################################
for key, value in powerbi_paths.items():
    response_content=get_response(base_urls.get('powerbi'), value)
    if (response_content.status_code == 200):
        dfResponse=response_to_df(response_content)
        dfResponse=dfResponse.select(explode("value")) \
            .select("col.*"
                )
        createTable(dfResponse, key)

In [ ]:
# write raw data to a json file
# Not required for now
'''
requestDate = datetime.datetime.now()
outputFilePath = "//lakehouse/default/Files/refreshables/top_200_{year}{month}{day}.json".format(year = requestDate.strftime("%Y"), month= requestDate.strftime("%m"), day=requestDate.strftime("%d"))
 
if not os.path.exists(os.path.dirname(outputFilePath)):    
    print("creating folder: " + os.path.dirname(outputFilePath))    
    os.makedirs(os.path.dirname(outputFilePath), exist_ok=True)
 
with open(outputFilePath, 'wb') as f:
    f.write(refreshables_response.content)
'''